In [ ]:
from datetime import datetime
from pathlib import Path

import pandas as pd

In [ ]:
data_path = Path("../data")

columns_of_interest = ["Datum", "Typ", "Wert", "Buchungswährung"]
how_many_types_expected = 5

type_mapping = {
    # Map Mintos types to Portfolio Performance types
    "Tilgungszahlungen": None,
    "Zinszahlungen": "Zinsen",
    "Erhaltene Zinsen aus Kreditrückkauf": "Zinsen",
    "Erhaltene Tilgung aus Kreditrückkauf": None,
    "Übertragungsabgleich der Verzugszinseinnahmen": "Zinsen",
    "Investitionen in Darlehen": None,
    "Zinseinnahmen aus ausstehenden Zahlungen": "Zinsen",
    "Steuereinbehalt": "Steuern",
    "Erhaltene Tilgung aus Rückkauf kleiner Kreditteile": None,
    "Zweitmarkt-Transaktion – Ab- oder Aufschlag": "Zinsen",  # noqa: RUF001
    "Zweitmarkt-Transaktion": None,
    "Zweitmarktgebühr": "Gebühren",
    "Einzahlungen": "Einlage",
    "Abhebung": "Entnahme",
}

In [ ]:
statement_files = list(data_path.glob("*.csv"))

[x.name for x in statement_files]

In [ ]:
data = (
    pd.concat([pd.read_csv(x) for x in list(data_path.glob("*.csv"))])
    .convert_dtypes()
    .assign(
        Datum=lambda df: pd.to_datetime(df["Datum"]).dt.date,
        Typ=lambda df: df["Zahlungsart"].replace(type_mapping),
        Wert=lambda df: df.groupby([df["Datum"], "Typ"])["Umsatz"].transform("sum").round(5),
    )
    .rename(columns={"Währung": "Buchungswährung"})
    .dropna(subset="Typ")
    .drop_duplicates(columns_of_interest)
)

In [ ]:
if len(data["Typ"].unique().tolist()) != how_many_types_expected:
    raise Exception(f"Expecting {how_many_types_expected} distinct values here..")


data["Typ"].unique().tolist()

In [ ]:
data[columns_of_interest].to_csv(
    data_path / f"{datetime.now().strftime('%Y-%m-%d')}.csv",
    index=False,
    sep=";",
    decimal=",",
)